In [1]:
%matplotlib inline
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import os
cuda = torch.cuda.is_available()
cuda =False
device = 'cuda' if cuda else 'cpu'

In [2]:
!git clone https://github.com/Sibylse/softmaxKmeans.git

fatal: destination path 'softmaxKmeans' already exists and is not an empty directory.


In [3]:
%cd softmaxKmeans
!git pull

/Users/shess/Scripts/softmaxKmeans
remote: Enumerating objects: 303, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 303 (delta 162), reused 287 (delta 153), pack-reused 2
Receiving objects: 100% (303/303), 319.73 MiB | 7.95 MiB/s, done.
Resolving deltas: 100% (162/162), completed with 2 local objects.
From https://github.com/Sibylse/softmaxKmeans
   632684c..ffae235  master     -> origin/master
 * [new branch]      multigpu   -> origin/multigpu
Updating 632684c..ffae235
Fast-forward
 checkpoint/Cifar10ResNetGauss.t7                   | Bin 44775307 -> 44775307 bytes
 checkpoint/Cifar10_no_gp_ResNetGauss.t7            | Bin 0 -> 44775307 bytes
 checkpoint/MNISTLeNetGauss.t7                      | Bin 142084 -> 142212 bytes
 checkpoint/MNISTLeNetLinear.t7                     | Bin 142084 -> 142212 bytes
 checkpoint/MNISTd2LeNetGauss.t7                    | Bin 0 -> 137988 bytes
 checkpoint/MNISTd2LeNetGauss_DUQ.t7   

In [4]:
import train
from train.models import *
from train import *
from train.layers import *
from train.loss import *
from train.optimization import Optimizer
from attack import FGSM

In [24]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
c=10

# Data
print('==> Preparing data..')
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

testset = datasets.MNIST(root='./data', train=False, download=True, transform=trans)
# Select only some classes for motivating picture
idx = testset.targets < c
testset.targets = testset.targets[idx]
testset.data = testset.data[idx]
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)

==> Preparing data..


In [6]:
path = "checkpoint/"
def load_net(name,dim_embedding,classifier):
    net = LeNet(dim_embedding,classifier)
    checkpoint = torch.load(path+name+'.t7',map_location='cpu')
    net.load_state_dict(checkpoint['net'])
    print('LeNet ACC:',checkpoint['acc'])
    return net

In [8]:
d=10

In [9]:
net_dict={}
net_dict["sm"] = load_net('MNISTLeNetLinear',d,nn.Linear(in_features = d, out_features = c))
#net_dict["sm_gp"] = load_net('MNISTLeNetLinear_gp',2,nn.Linear(in_features = 2, out_features = c))
net_dict["duq"] = load_net('MNISTLeNetGauss_DUQ',d,Gauss_DUQ(in_features = d, out_features = c, gamma=10))
net_dict["ga"] = load_net('MNISTLeNetGauss',d,Gauss(in_features = d, out_features = c, gamma=10))
#net_dict["ga_mv"] = load_net('MNISTLeNetGauss_MV',2,Gauss_MV(in_features = 2, out_features = c, gamma=10))

LeNet ACC: tensor(99.2600)
LeNet ACC: tensor(98.4767)
LeNet ACC: tensor(99.3000)


In [10]:
criterion_dict = {}
criterion_dict["sm"] = CE_Loss(net_dict["sm"].classifier, c, device)
#criterion_dict["sm_gp"] = CE_Loss(net_dict["sm_gp"].classifier, c, device)
criterion_dict["ga"] = CE_GALoss(net_dict["ga"].classifier, c, device)
#criterion_dict["ga_mv"] = BCE_GALoss(net_dict["ga_mv"].classifier, c, device)
criterion_dict["duq"] = BCE_DUQLoss(net_dict["duq"].classifier, c, device)

# Attack

In [11]:
!pip install --upgrade git+https://github.com/BorealisAI/advertorch.git

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Cloning https://github.com/BorealisAI/advertorch.git to /private/var/folders/3h/v9rwrxsj02n4bvnfdw3s81vc0000gn/T/pip-req-build-x3cdepzr
  Running command git clone --filter=blob:none --quiet https://github.com/BorealisAI/advertorch.git /private/var/folders/3h/v9rwrxsj02n4bvnfdw3s81vc0000gn/T/pip-req-build-x3cdepzr
  Resolved https://github.com/BorealisAI/advertorch.git to commit e063e341c87d9a621ae1a8f72c3507d5ea5bd327
  Preparing metadata (setup.py) ... done
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


## Carlini & Wagner Attack

In [12]:
from advertorch.attacks import CarliniWagnerL2Attack

In [13]:
net_dict.keys()

dict_keys(['sm', 'duq', 'ga'])

In [25]:
inputs, targets = next(iter(testloader))

In [27]:
cw_dict, examples_dict={},{}
for net_key in net_dict.keys():
    cw = CarliniWagnerL2Attack(net_dict[net_key], c, confidence=-np.log(0.2), max_iterations=100, clip_min=-0.5, clip_max=0.5 )
    examples_dict[net_key]= cw.perturb(inputs, targets)
    criterion = criterion_dict[net_key]
    conf,pred = criterion.conf(net_dict[net_key].embed(examples_dict[net_key])).max(1)
    print(criterion.__class__.__name__,((targets != pred) & (conf>0.1)).sum())

CE_Loss tensor(402)
BCE_DUQLoss tensor(41)
CE_GALoss tensor(1)


# One Pixel Attack

In [28]:
from advertorch.attacks import SinglePixelAttack
cw_dict, examples_dict={},{}
for net_key in net_dict.keys():
    criterion = criterion_dict[net_key]
    attack = SinglePixelAttack(net_dict[net_key].embed, max_pixels=1, loss_fn=criterion, clip_min=-0.5, clip_max=0.5)
    examples_dict[net_key]= attack.perturb(inputs, targets)
    conf,pred = criterion.conf(net_dict[net_key].embed(examples_dict[net_key])).max(1)
    print(criterion.__class__.__name__,((targets != pred) & (conf>0.1)).sum())


CE_Loss tensor(14)
BCE_DUQLoss tensor(2)
CE_GALoss tensor(1)


# FGSM attack

In [29]:
from advertorch.attacks import GradientSignAttack
cw_dict, examples_dict={},{}
for net_key in net_dict.keys():
    criterion = criterion_dict[net_key]
    attack = GradientSignAttack(net_dict[net_key].embed, eps=0.1, loss_fn=criterion, clip_min=-0.5, clip_max=0.5)
    examples_dict[net_key]= attack.perturb(inputs, targets)
    conf,pred = criterion.conf(net_dict[net_key].embed(examples_dict[net_key])).max(1)
    print(criterion.__class__.__name__,((targets != pred) & (conf>=0.1)).sum())

CE_Loss tensor(130)
BCE_DUQLoss tensor(41)
CE_GALoss tensor(29)


In [32]:
examples_dict["ga"].shape

torch.Size([1000, 1, 28, 28])

# Linf PGD Attack

In [21]:
from advertorch.attacks import LinfPGDAttack
cw_dict, examples_dict={},{}
for net_key in net_dict.keys():
    criterion = criterion_dict[net_key]
    attack = LinfPGDAttack(net_dict[net_key].embed, loss_fn=criterion, eps=0.1, nb_iter=40, eps_iter=0.01, clip_min=-0.5, clip_max=0.5)
    examples_dict[net_key]= attack.perturb(inputs, targets)
    conf,pred = criterion.conf(net_dict[net_key].embed(examples_dict[net_key])).max(1)
    print(criterion.__class__.__name__,((targets != pred) & (conf>=0.1)).sum())


CE_Loss tensor(24)
BCE_DUQLoss tensor(45)
CE_GALoss tensor(14)


# L2PGDAttack

In [22]:
from advertorch.attacks import L2PGDAttack
cw_dict, examples_dict={},{}
for net_key in net_dict.keys():
    criterion = criterion_dict[net_key]
    attack = L2PGDAttack(net_dict[net_key].embed, loss_fn=criterion, eps=0.4, nb_iter=40, eps_iter=0.01, clip_min=-0.5, clip_max=0.5)
    examples_dict[net_key]= attack.perturb(inputs, targets)
    conf,pred = criterion.conf(net_dict[net_key].embed(examples_dict[net_key])).max(1)
    print(criterion.__class__.__name__,((targets != pred) & (conf>0.1)).sum())


NameError: name 'L2PGDAttack' is not defined

In [43]:
cw_dict, examples_dict={},{}
for net_key in net_dict.keys():
    criterion = criterion_dict[net_key]
    attack = L2PGDAttack(net_dict[net_key].embed, loss_fn=criterion, eps=0.4, nb_iter=40, eps_iter=0.01, clip_min=-0.5, clip_max=0.5)
    examples_dict[net_key]= attack.perturb(inputs, targets)
    conf,pred = criterion.conf(net_dict[net_key].embed(examples_dict[net_key])).max(1)
    print(criterion.__class__.__name__,((targets != pred) & (conf>0.1)).sum())


CE_Loss tensor(1)
CE_Loss tensor(7)
BCE_DUQLoss tensor(2)
BCE_GALoss tensor(0)
BCE_GALoss tensor(0)


# Sparse L1 Descent Attack

In [23]:
from advertorch.attacks import SparseL1DescentAttack
adv_dict, examples_dict={},{}
for net_key in net_dict.keys():
    print(net_key)
    criterion = criterion_dict[net_key]
    adversary = SparseL1DescentAttack(predict=net_dict[net_key].embed, eps=0.3, loss_fn=criterion, clip_min=-0.5, clip_max=0.5)
    examples_dict[net_key]= adversary.perturb(inputs, targets)
    conf,pred = criterion.conf(net_dict[net_key].embed(examples_dict[net_key])).max(1)
    print(criterion.__class__.__name__,((targets != pred) & (conf>0.1)).sum())


sm
CE_Loss tensor(0)
duq
BCE_DUQLoss tensor(0)
ga
CE_GALoss tensor(0)


In [ ]:
input, output = next(iter(testloader))
torch.min(input), torch.max(input)

# Plot examples
Doesn't work as is, is still adapted to the epsilons

In [ ]:
# Plot several examples of adversarial samples at each epsilon
def plot_mnist_adv(net, criterion, examples, min_conf=0.1):
    plt.figure(figsize=(8,15))

    conf_pert, pred_pert = criterion.conf(net.embed(examples)).max(1)
    idx = (conf_pert>min_conf)
    examples, conf_pert, pred_pert = examples[idx], conf_pert[idx], pred_pert[idx]
    for j in range(min(len(examples),8)):
        plt.subplot(1,8,j+1)
        plt.xticks([], [])
        plt.yticks([], [])
        plt.title("{} ({:.3f})".format(pred_pert[j], conf_pert[j]))
        plt.imshow(examples[j].squeeze().detach().numpy(), cmap="gray")
    plt.tight_layout()
    plt.show()

In [ ]:
net_key = "ga"
plot_mnist_adv(net_dict[net_key],criterion_dict[net_key],examples_dict[net_key])